In [1]:
import pandas as pd
from pandarallel import pandarallel
from typing import defaultdict
from datetime import datetime

In [2]:
data_path = '../data/'

### ИПЦ

In [3]:
ipc = pd.read_feather(data_path + '/external/ИПЦ.feather')

In [4]:
ipc.drop(columns=[(2019, 'II квартал')], inplace=True)

In [5]:
ipc.columns = ipc.columns.to_flat_index()
ipc.reset_index(inplace=True)
ipc.rename(columns={'level_0': 'area', 'level_1': 'subject'}, inplace=True)
months_mapper = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}
ipc.columns = ['area', 'subject'] + [pd.to_datetime(str(col[0]) + '-' + str(months_mapper[col[1]]), errors='coerce').date()
                                     for col in ipc.iloc[:, 2:].columns]

In [6]:
ipc = pd.melt(ipc, id_vars=ipc.columns[:2], value_vars=ipc.columns[2:], var_name='month', value_name='ipc')

In [7]:
print(ipc.shape)

(56588, 4)


In [8]:
ipc.area.nunique()

41

In [9]:
def columns_report(data: pd.DataFrame) -> None:

    print('Rows:', data.shape[0], end='\n'*2)

    stats = defaultdict(list)
    for col in data.columns:
        stats['Column'].append(col)
        stats['Unique'].append(data[col].nunique())
        stats['Duplicates'].append(data[col].duplicated().sum())
        stats['Missing'].append(data[col].isna().sum())
        stats['Missing%'].append(data[col].isna().mean() * 100)
        stats['HitRate%'].append((1 - data[col].isna().mean()) * 100)
    stats = pd.DataFrame(stats)

    print(stats.to_string(index=False))

In [10]:
columns_report(ipc)

Rows: 56588

 Column  Unique  Duplicates  Missing  Missing%   HitRate%
   area      41       56547        0  0.000000 100.000000
subject      15       56573        0  0.000000 100.000000
  month      94       56494        0  0.000000 100.000000
    ipc    1409       55178     3069  5.423411  94.576589


In [11]:
areas = ['край', 'область', 'республика', 'округ', 'российская федерация', 'москва', 'санкт-петербург']
stop_words = areas[:4]

In [12]:
def rm_stop_words(s: str, stop_words=stop_words):
    return ' '.join(w for w in s.split() if w not in stop_words)

def prep_fedstat_geo(data: pd.DataFrame, subs_to_maintain: list,
                     geo_col: str = 'area') -> pd.DataFrame:

    pandarallel.initialize(progress_bar=False, nb_workers=12)

    data[geo_col] = data[geo_col]\
        .str.strip()\
        .str.lower()\
        .str.replace(r'\s*\(.*\)', '', regex=True)\
        .str.replace(r' - .*', '', regex=True)
    data = data.loc[
        data[geo_col].str.contains('|'.join(subs_to_maintain))].reset_index(drop=True)
    data[geo_col] = data[geo_col].parallel_apply(
        rm_stop_words)\
        .str.strip()

    return data

In [13]:
ipc = prep_fedstat_geo(ipc, areas)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [14]:
ipc.loc[ipc.area == 'город москва столица российской федерации город федерального значения', 'area'] = 'москва'
ipc.loc[ipc.area == 'город санкт-петербург город федерального значения', 'area'] = 'санкт-петербург'

In [20]:
ipc.to_feather(data_path + 'external/ipc_preprocessed.frt')

d:\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [23]:
ipc.head()

,area,subject,month,ipc
0,российская федерация,Все товары и услуги,2016-01-01,100.96
1,российская федерация,Базовый индекс потребительских цен,2016-01-01,100.82
2,российская федерация,Оплата холодного водоснабжения и водоотведения,2016-01-01,100.09
3,российская федерация,Оплата горячего водоснабжения,2016-01-01,100.06
4,российская федерация,Услуги по подключению к сети Интернет,2016-01-01,100.02


### Ставка ЦБ